In [3]:
import pandas as pd

# load the sales dataframe created in the previous load_salesdata notebook
sales = pd.read_csv("processed_data/sales_2021-2022_NEW_CATEGORIES.csv", parse_dates=True)
sales["date"] = pd.to_datetime(sales["date"])
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 0 to 686
Columns: 175 entries, date to _count
dtypes: datetime64[ns](1), float64(163), int64(9), object(2)
memory usage: 939.4+ KB


#### Define the relevant weather variables and load them into the existing sales dataframe

In [4]:
weather_variables = {"precipitation": "RR", 
                    "temperature_mean": "TG", 
                    "temperature_max": "TX",
                    "temperature_min": "TN",
                    "sunshine_duration": "SS",
                    "cloud_cover": "CC",
                    "wind_speed": "FG",
                    "humidity": "HU",
                    "pressure": "PP",
                    "snow_depth": "SD"}

sales_and_weather = sales.copy()

for variable, code in weather_variables.items():
    weather_var = pd.read_csv(f"raw_weatherdata\{code}_STAID002763.txt", sep=",", skiprows=19, parse_dates=True)
    
    # clean the data, drop unnecessary columns, rename columns, 
    # convert date to datetime format, drop all dates before 2020-01-01, 
    # drop all rows with missing values and drop the quality column
    weather_var.columns = weather_var.columns.str.strip()
    weather_var.drop(columns=["SOUID"], inplace=True)
    weather_var.rename(columns={"DATE": "date"}, inplace=True)
    weather_var["date"] = pd.to_datetime(weather_var["date"], format="%Y%m%d")
    weather_var = weather_var[weather_var["date"] >= "2020-01-01"]
    weather_var.dropna(inplace=True)
    weather_var = weather_var[weather_var[f"Q_{code}"] == 0]
    weather_var = weather_var[["date", code]]

    # merge the weatherdata into the sales dataframe
    sales_and_weather = sales_and_weather.merge(weather_var, on="date", how="left")
    sales_and_weather.rename(columns={code: variable}, inplace=True)

sales_and_weather.to_csv("processed_data/sales_and_weather_2021-2022.csv", index=False)
sales_and_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687 entries, 0 to 686
Columns: 185 entries, date to snow_depth
dtypes: datetime64[ns](1), float64(173), int64(9), object(2)
memory usage: 998.3+ KB
